# SIC PROJECT



In [8]:
#requirements and dependencies
#check if the requirements are installed
requirements = ["tensorflow", "numpy", "cv2", "tqdm", "matplotlib", "tensorboard", "tensorflow_hub", "sklearn"]
#if not, quit
for requirement in requirements:
    try:
        __import__(requirement)
    except ImportError:
        print("You need to install the following package: " + requirement)
        print("You can do this by running: pip install " + requirement)
        quit()
print("All requirements are installed")

All requirements are installed


In [9]:
#import packages
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tqdm
import datetime
from sklearn.model_selection import train_test_split

import tensorboard
%load_ext tensorboard
print("success")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
success


In [10]:
#check metal gpu
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

/device:GPU:0
Num GPUs Available:  1


2023-07-31 05:09:47.350444: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-31 05:09:47.350483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
#load data
def load_data(path, IMG_SIZE=224):
    """
    Load image data from directory, resize and normalize it.
    return tuple `(images, labels)`.
    """
    categories = os.listdir(path)
    data = []
    labels = []
    for category in tqdm.tqdm(categories):
        folder_path = os.path.join(path, category)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                data.append(img)
                labels.append(categories.index(category))

    return np.array(data), tf.keras.utils.to_categorical(labels)

In [12]:
#built from scratch model
def get_model(IMG_SIZE = 224, NUM_CLASSES = 3):
    
    #model architecture
    model = tf.keras.models.Sequential([

       tf.keras.layers.Conv2D(
           64, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)
       ),

       tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

       tf.keras.layers.Dropout(0.25),

       tf.keras.layers.Conv2D(
           128, (3, 3), activation="relu"),

       tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

       tf.keras.layers.Dropout(0.25),

       tf.keras.layers.Flatten(),

       tf.keras.layers.Dense(128, activation="relu"),

       tf.keras.layers.Dropout(0.25),
       ##

       tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")

    ])

    # Compile Model
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["categorical_accuracy"]
    )

    return model

In [13]:
#load data
train_data, train_labels = load_data('Train')
test_data, test_labels = load_data('Test')

#load model
model = get_model()

#some parameters
EPOCHS = 1
IMG_SIZE = 224
BATCH_SIZE = 1

#for tensorboard
log_dir = "logs/fit/" + str(len(os.listdir('logs/fit')) + 1) if os.path.exists('logs/fit') else "logs/fit/1"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,  histogram_freq=1)

#train and validate the model
history = model.fit(x = train_data, y = train_labels, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[tensorboard_callback])
#evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels, callbacks=[tensorboard_callback])

100%|██████████| 3/3 [00:00<00:00,  8.12it/s]


   1/2429 [..............................] - ETA: 13:29 - loss: 0.0000e+00 - categorical_accuracy: 1.0000

2023-07-31 05:09:51.212654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2429/2429 [==============================] - ETA: 0s - loss: 105.9439 - categorical_accuracy: 0.4854

2023-07-31 05:11:13.405641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2429/2429 [==============================] - 100s 41ms/step - loss: 105.9439 - categorical_accuracy: 0.4854 - val_loss: 1.3437 - val_categorical_accuracy: 0.0000e+00


2023-07-31 05:11:31.331977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 1s 47ms/step - loss: 1.1129 - categorical_accuracy: 0.3267


In [ ]:
#lunch tensorboard to plot the results and the graphs
print("model accuracy: " + str(test_acc))
print("allah y3een")
%tensorboard --logdir=logs --port=6969